In [5]:
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats

import random
import math
import pickle
from collections import OrderedDict
import time
import os

import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.utils.data
from torch.nn.parameter import Parameter
from torch.nn import init

from tqdm import tqdm

In [6]:
class ResBlock(nn.Module):
    def __init__(self, channels, dilation):
        super(ResBlock, self).__init__()
        self.conv1 = nn.Conv1d(channels, channels, kernel_size=3, stride=1, padding=dilation, dilation=dilation)
        self.conv1_bn = nn.BatchNorm1d(channels)
        self.conv2 = nn.Conv1d(channels, channels, kernel_size=3, stride=1, padding=dilation, dilation=dilation)
        self.conv2_bn = nn.BatchNorm1d(channels)
        
    def forward(self, x):
        x1 = F.relu( self.conv1_bn( self.conv1(x) ) )
        return self.conv2_bn( self.conv2(x1) ) + x

class EmbedderNet(nn.Module):
    def __init__(self, length, channels, outchannels):
        super(EmbedderNet, self).__init__()
        self.length = length
        
        self.pool = nn.MaxPool1d(2, 2, ceil_mode = True)
        self.conv = nn.Conv1d(40, channels, 1, 1, 0)
        self.conv_bn = nn.BatchNorm1d(channels)
        
        self.block1 = ResBlock(channels, 1)
        self.block2 = ResBlock(channels, 1)
        self.block3 = ResBlock(channels, 1)
        self.block4 = ResBlock(channels, 1)
        self.block5 = ResBlock(channels, 1)
        
        self.embed_1 = nn.Linear( int( channels*math.ceil(math.ceil(self.length/2)/2) ) , 128)
        self.embed_2 = nn.Linear(128, outchannels)
        
    def forward(self, x):
        batchsize = x.shape[0]
        x = F.relu( self.conv_bn( self.conv(x) ) )
        
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.pool( self.block4(x) )
        x = self.pool( self.block5(x) ).view(batchsize,-1)
        return self.embed_2( F.relu( self.embed_1(x) ) )
    
class Predictor_Dot(nn.Module):
    def __init__(self):
        super(Predictor_Dot, self).__init__()
        self.embedPrefix = EmbedderNet(10, 64, 16)
        self.embedSuffix = EmbedderNet(10, 64, 16)
        
    def forward(self, x):
        prefix = x[:,:,:10]
        suffix = x[:,:,10:]
        return torch.sum(self.embedPrefix(prefix) * self.embedSuffix(suffix), dim = 1)

In [7]:
class NetEnsemble(nn.Module):
    def __init__(self, lst):
        super(NetEnsemble, self).__init__()
        self.nets = nn.ModuleList(lst)
        
    def forward(self, x):
        return torch.stack([net(x) for net in self.nets]).permute((1,0,2))

class Combine(nn.Module):
    def __init__(self):
        super(Combine, self).__init__()
        
    def forward(self, x1, x2):
        return torch.einsum("nij,nij->n",x1,x2)
        
    def getPrefix(self, x1, x2):
        return torch.einsum("nij,ij->n",x1,x2)
    
    def getSuffix(self, x1, x2):
        return torch.einsum("ij,nij->n",x1,x2)

In [21]:
# Pass in a list of models to get an ensemble
def loadModels(fns):
    prefixMaps = []
    suffixMaps = []
    for fn in fns:
        net = Predictor_Dot()
        net.load_state_dict(torch.load( fn, map_location='cpu' ))
        prefixMaps.append(net.embedPrefix)
        suffixMaps.append(net.embedSuffix)
    prefixEnsemble = NetEnsemble(prefixMaps)
    suffixEnsemble = NetEnsemble(suffixMaps)
    combine = Combine()
    prefixEnsemble.eval().cuda()
    suffixEnsemble.eval().cuda()
    combine.eval()
    return prefixEnsemble, suffixEnsemble, combine

def calibrate(model):
    with open("../model_training/encoding.pkl", 'rb') as fin:
        encoding = pickle.load(fin)
    prefix = []
    suffix = []
    with open("calibration.txt", 'rt') as fin:
        for line in fin:
            line = line.rstrip('\n')
            line = [encoding[c] for c in line]
            prefix.append(np.array(line[:10]).T)
            suffix.append(np.array(line[10:]).T)
    
    calibrationset = torch.utils.data.TensorDataset(
        torch.tensor(np.stack(prefix), dtype = torch.float32),
        torch.tensor(np.stack(suffix), dtype = torch.float32))
    loader = torch.utils.data.DataLoader(calibrationset, batch_size=10000, shuffle=False, num_workers=2, drop_last=False)
    
    outputs = []
    prefixModel, suffixModel, combine = model
    with torch.no_grad():
        for prefix, suffix in tqdm(loader, position = 0, leave = True):
            outputs.append( combine( prefixModel(prefix.cuda()), suffixModel(suffix.cuda()) ).cpu().numpy() )
    return np.std(np.concatenate(outputs), ddof = 1)

In [25]:
def getTranslate():
    with open("../model_training/encoding.pkl", 'rb') as fin:
        onehot = pickle.load(fin)
    dic = {}
    for k in onehot:
        dic[k] = torch.tensor(onehot[k]).float().cuda()
    return dic

class runningSum():
    def __init__(self, embedShape):
        self.x = torch.zeros(*embedShape).double()
        self._c = torch.zeros(*embedShape).double()
        
    def add(self, y):
        self.x = self.x + y

class Translator():
    def __init__(self, translate, model):
        self.SymbolToCode = translate
        self.NumberToLetter = [aa for aa in self.SymbolToCode]
        self.NumberToCode = []
        self.SymbolToNumber = {}
        self.CodeToEmbedding = model
        self.pad = -1
        for i, aa in enumerate(self.NumberToLetter):
            self.SymbolToNumber[aa] = i
            self.NumberToCode.append( self.SymbolToCode[aa] )
            if aa == 'J':
                self.pad = i
            
    def toString(self, seq):
        return ''.join([self.NumberToLetter[i] for i in seq])

class Sequence():
    def __init__(self, sequence, translator):
        self.translator = translator
        self.sequence = [self.translator.SymbolToNumber[c] for c in sequence]
        self.code = torch.stack( [self.translator.NumberToCode[c] for c in self.sequence] )
        self.tracker = None
        self.runningSum = None
        self.embedding = None
            
    def register(self, tracker):
        self.tracker = tracker
        for i,j in enumerate(self.sequence):
            self.tracker.counts[i][j] += 1
        self.tracker.seqs.add( self.translator.toString(self.sequence) )
        
    def register2(self, runningSum):
        self.runningSum = runningSum
        self.runningSum.add(self.embedding)
        
    def update(self, position, aa_number, new_embedding):
        self.tracker.seqs.remove( self.translator.toString(self.sequence) )
        self.tracker.counts[position][ self.sequence[position] ] -= 1
        self.sequence[position] = aa_number
        self.tracker.counts[position][ aa_number ] += 1
        self.tracker.seqs.add( self.translator.toString(self.sequence) )
        
        self.code[position] = self.translator.NumberToCode[aa_number]
        with torch.no_grad():
            self.runningSum.add(-self.embedding.double())
            self.embedding = new_embedding
            self.runningSum.add(self.embedding.double())
            
    def isFree(self, position, proposals):
        current = self.sequence[position]
        freeMove = []
        for i in proposals:
            self.sequence[position] = i
            freeMove.append(self.translator.toString(self.sequence) not in self.tracker.seqs)
        self.sequence[position] = current
        return freeMove
            
    # Generate a list of embeddings corresponding to samples that can then be used to calculate deltas upstream
    def mockUpdates(self, position):
        samples = []
        current = self.sequence[position]
        for i in range(len(self.translator.NumberToCode)):
            if i != current and i != self.translator.pad:
                self.sequence[position] = i
                samples.append(i)
        self.sequence[position] = current
        if len(samples) == 0:
            return [], []
        
        seqs = torch.stack( [self.code for i in range(len(samples))] )
        for i,proposal in enumerate(samples):
            seqs[i, position] = self.translator.NumberToCode[proposal]
            
        return seqs, samples  
        
class FrequencyTracker():
    def __init__(self, sequences, length, sigma):
        self.counts = np.zeros( (length, sigma) )
        self.seqs = set()
        for seq in sequences:
            seq.register(self)
        
class SubLibrary():
    def __init__(self, sequences, translate, model, updateParam, length):
        sigma = len(translate)
        
        self.translator = Translator(translate, model)
        self.seqs = [Sequence(seq, self.translator) for seq in sequences]
        self.size = len(self.seqs)
        self.updateParam = updateParam
        
        with torch.no_grad():
            embeds = self.translator.CodeToEmbedding( torch.stack([seq.code for seq in self.seqs]).permute(0,2,1) ).cpu()
            for i, seq in enumerate(self.seqs):
                seq.embedding = embeds[i].detach()
        
        partition = {}
        for seq, update in zip(self.seqs, self.updateParam):
            if update not in partition:
                partition[update] = []
            partition[update].append(seq)
        
        self.frequencyTrackers = {}
        for k in partition:
            self.frequencyTrackers[k] = FrequencyTracker(partition[k], length, sigma)
        
        self.embeddingSum = runningSum(self.seqs[0].embedding.shape)
        for seq in self.seqs:
            seq.register2(self.embeddingSum)
            
    def _entropy(self, arr, z):
        return -np.log(np.maximum(1,arr)/z) * (arr/z)
    
    def dedupe(self, seqindex, position, ls):
        ls2 = []
        proposals = ls[0]
        isFree = self.seqs[seqindex].isFree(position, proposals)
        for arr in ls:
            ls2.append([z for z,x in zip(arr,isFree) if x])
        return ls2
    
    def getEntropyChanges(self, counts, current, proposals):
        x = counts[current]
        z = np.sum(counts)
        h_now = self._entropy(counts, z)
        dh = self._entropy(counts[current]-1, z) - h_now[current]
        deltah = (self._entropy(counts+1, z) - h_now) + dh
        return np.take(deltah, proposals) * z
    
    def proposeEveryChange(self, seqindex, position):
        codes, proposals = self.seqs[seqindex].mockUpdates(position)
        if len(proposals) == 0: return [], None, []
        return codes, self.seqs[seqindex].embedding, proposals
    
    def getDeltaEntropy(self, seqindex, position, proposals):
        deltaEntropy = self.getEntropyChanges(self.seqs[seqindex].tracker.counts[position],
                                              self.seqs[seqindex].sequence[position],
                                              proposals)
        return deltaEntropy
    
    def update(self, seqindex, position, aa_number, new_embedding):
        self.seqs[seqindex].update(position, aa_number, new_embedding)
        
    def getEntropyScores(self):
        report = []
        for k in self.frequencyTrackers:
            counts = self.frequencyTrackers[k].counts
            z = np.sum(counts[0])
            h = np.array( [np.sum( self._entropy(arr, z) ) for arr in counts] )
            report.append( (z,h) )
        return report
    
    def getLib(self):
        return [self.translator.toString(z.sequence) for z in self.seqs]

class Optimizer():
    def __init__(self, prefix_seed, suffix_seed, updatePrefix, updateSuffix, prefixLength, suffixLength, translate, model, std):
        prefixModel, suffixModel, combine = model
        self.prefixLib = SubLibrary(prefix_seed, translate, prefixModel, updatePrefix, prefixLength)
        self.suffixLib = SubLibrary(suffix_seed, translate, suffixModel, updateSuffix, suffixLength)
        self.combine = combine
        self.entropyWeight = 0.01
        self.T = 50
        self.std = std
        self.updatePrefix = updatePrefix
        self.updateSuffix = updateSuffix
        self.prefixLength = prefixLength
        self.suffixLength = suffixLength
        
    def sweep(self):
        psweep = [i for i in range(self.prefixLength)]
        ssweep = [i for i in range(self.suffixLength)[::-1]]
        for p,s in zip(psweep, ssweep):
            pcol = self.sweepColumn(p, self.updatePrefix, self.prefixLib)
            scol = self.sweepColumn(s, self.updateSuffix, self.suffixLib)
            for i in range(max(len(pcol), len(scol))):
                if i < len(pcol):
                    self.executeUpdate(True, pcol[i], self.prefixLib, self.suffixLib, p)
                if i < len(scol):
                    self.executeUpdate(False, scol[i], self.suffixLib, self.prefixLib, s)
            pcol = []
            scol = []
            torch.cuda.empty_cache()
                    
    def executeUpdate(self, isPrefix, command, lib, otherlib, position):
        seqindex, dEmbed, cEmbed, proposals = command
        with torch.no_grad():
            dScore = (self.combine.getPrefix(dEmbed-cEmbed, self.suffixLib.embeddingSum.x.float()) if isPrefix
                      else self.combine.getSuffix(self.prefixLib.embeddingSum.x.float(), dEmbed-cEmbed)).numpy()
            
        proposals, dEmbed, dScore = lib.dedupe( seqindex, position, (proposals, dEmbed, dScore) )
        dScore = np.array(dScore)
        if len(proposals) == 0: return
        
        dEntropy = lib.getDeltaEntropy(seqindex, position, proposals)
        dScore = (dScore/ (otherlib.size*self.std)) + (dEntropy * self.entropyWeight)
        
        dScore = np.concatenate( ((dScore * self.T), [0]) )
        proposals.append(-1)
        newIndex = random.choices(range(len(proposals)), np.exp(dScore - np.max(dScore)))[0]
        if proposals[newIndex] != -1:
            lib.update(seqindex, position, proposals[newIndex], dEmbed[newIndex].detach())
            
    def sweepColumn(self, position, updates, lib):
        index = 0
        commands = []
        bigTensor = []
        for seqindex, (mn, mx) in enumerate( updates ):
            if mn <= position < mx:
                codeProposals, cEmbed, proposals = lib.proposeEveryChange(seqindex, position)
                if len(proposals) == 0: continue
                
                rng = (index, index + len(proposals))
                commands.append( (seqindex, cEmbed, proposals, rng) )
                bigTensor.append(codeProposals)
                index += len(proposals)
        if len(bigTensor) == 0: return []
        with torch.no_grad():
            outTensor = lib.translator.CodeToEmbedding( torch.cat(bigTensor, dim = 0).permute((0,2,1)).cuda() )
        commands2 = []
        for i, (seqindex, cEmbed, proposals, rng) in enumerate(commands):
            dEmbed = outTensor[rng[0]:rng[1]].detach().cpu()
            commands2.append((seqindex, dEmbed, cEmbed, proposals))
        return commands2
        
    def getScore(self):
        with torch.no_grad():
            score = self.combine(self.prefixLib.embeddingSum.x.float().unsqueeze(0),
                                 self.suffixLib.embeddingSum.x.float().unsqueeze(0)).cpu().view(-1).item()/self.std
        h1 = self.prefixLib.getEntropyScores()
        h2 = self.suffixLib.getEntropyScores()
        ht1 = np.sum([z*h for z,h in h1]) * self.suffixLib.size
        ht2 = np.sum([z*h for z,h in h2]) * self.prefixLib.size
        ht = ht1 + ht2
        total = score + (ht*self.entropyWeight)
        return total
    
    def getLib(self):
        return self.prefixLib.getLib(), self.suffixLib.getLib()

In [19]:
def getSeed(n,mn,mx):
    aas = list("ACDEFGHIKLMNPQRSTVWYJ")
    sample = np.random.randint(0,20, (n*2, 10) )
    sample[:,:mn] = 20
    sample[:,10-mx:] = 20
    sample = [''.join( [aas[i] for i in s] ) for s in sample]
    sample = set(sample)
    if len(sample) < n:
        return getSeed(n,mn,mx)
    return list(sample)[:n]
    
def getSeedPref(n):
    seeds = []
    updates = []
    for i in range(0,7):
        seeds += getSeed(n,i,0)
        updates += [(i,10)]*n
    return seeds, updates

def getSeedSuff(n):
    seeds = []
    updates = []
    for i in range(0,7):
        seeds += getSeed(n,0,i)
        updates += [(0,10-i)]*n
    return seeds, updates

<h3>Generating a factorizable library</h3>

In [ ]:
# Initialize the ensemble with a list of models
ensembles = loadModels(["../model_training/weights/<MODEL WEIGHT FILE HERE>"])

# Determine the standard deviation of the ensemble
std = calibrate(ensembles)

In [26]:
# Initialize seed sequences
p,pu = getSeedPref(20)
s,su = getSeedSuff(20)

# Initialize the optimizer
opt = Optimizer(p, s, pu, su, 10, 10, getTranslate(), ensembles, std)

# Set entropy parameter lambda
opt.entropyWeight = 0.1

In [ ]:
# Initialize temperature
opt.T = 1

bestLib = None
bestScore = float("-inf")

# Optimize for 500 iterations
pbar = tqdm(range(500), position=0, leave=True)
for i in pbar:
    opt.sweep()
    torch.cuda.empty_cache()
    score = opt.getScore()
    if score > bestScore:
        bestLib = opt.getLib()
    pbar.set_description("Score = {:.2f}, Temperature = {:.5f}".format(score, 1/opt.T))
    
    # Lower the temperature every 5 iterations
    if i%5==4:
        opt.T *= 1.1
        
# Optimized segment libraries
prefixLibrary, suffixLibrary = bestLib